# Classifiez automatiquement des biens de consommation

## 1.1 Problématique

L’entreprise "Place de marché”, <br />
souhaite lancer une marketplace e-commerce.

Sur la place de marché, des vendeurs <br />
proposent des articles à des acheteurs <br />
en postant une **photo** et une **description**.

Pour l'instant, l'attribution de la catégorie<br /> 
d'un article est effectuée manuellement par <br />
les vendeurs et est donc peu fiable. <br />
De plus, le volume des articles est <br />
pour l’instant très petit.

Pour rendre l’expérience utilisateur <br />
des vendeurs (faciliter la mise en ligne <br />
de nouveaux articles) et des acheteurs (faciliter <br />
la recherche de produits) la plus fluide <br />
possible et dans l'optique d'un passage <br />
à l'échelle, il devient nécessaire d'automatiser <br />
cette tâche.

Il nous est demandé d'étudier la faisabilité <br />
d'un moteur de classification des articles <br />
en différentes catégories, avec un niveau <br />
de précision suffisant.

## 1.2 Objectifs dans ce projet

Réaliser une première **étude de faisabilité d'un moteur <br />
de classification d'articles** basé sur une <u>image</u> et une <br />
<u>description</u> pour l'automatisation de l'attribution <br />
de la *catégorie de l'article*.

**Analyser le jeu de données** en réalisant un <u>prétraitement <br />
des images et des descriptions</u> des produits, une <u>réduction <br />
de dimension</u>, puis un <u>clustering</u>.

Les résultats du clustering seront présentés sous la forme <br />
d’une **représentation en deux dimensions**, qui illustrera <br />
le fait que <u>les caractéristiques extraites permettent <br />
de regrouper des produits de même catégorie</u>.

# 2. Import des librairies et réglage de Pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pickle

# !pip install texthero
import texthero as hero

from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans, MiniBatchKMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import adjusted_rand_score

from keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras import Model

In [ ]:
# Changing Pandas settings for
# be able to display more rows and more columns.
pd.set_option("max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# 3. Déclaration des fonctions

J'ai décomposé en fonctions toutes les actions <br />
clés réalisés dans ce projet afin :<br />
 - d'améliorer la lecture du code et de ma démarche
 - de donner la possibilité à quiconque de facilement <br />
   réutiliser mon code pour ses propres projets.
   
<u>L'ensemble des fonctions que j'ai écrit</u> et utilisé <br />
dans ce projet, pour les parties concernant <br />
le traitement du texte, des images ou <br />
du texte + images <u>sont listés ci-dessous</u> :

In [ ]:
def first_product_category_tree(txtATraiter):
    '''
    Returns the first element of product_category_tree
    '''
    return txtATraiter.replace('["','')\
                      .replace('"]','')\
                      .replace(' >> ','>>')\
                      .split('>>')[0]

In [ ]:
def lemma_filter_english_words(txtToLemmatize,
                               toSentence=True,
                               tagFilter=None):
    '''
    This function lemmatizes the sentence given as argument.
    It returns the choice:
     - a lemmatized sentence
     - a list containing each term of the lematized sentence.
    In addition, it is possible to communicate a list containing 
    the nature or function of the words to be returned,
    the others will be ignored and deleted.
    The list of terms available for <tagFilter> are:
     - r': Adverb
     - n': Name
     - v': Verb
     '''
    from nltk.stem import WordNetLemmatizer
    from nltk import pos_tag, word_tokenize
    sentenceToReturn = []
    wnl = WordNetLemmatizer()
    
    # Tokenize et tag each word of the sentence
    for word, tag in pos_tag(word_tokenize(txtToLemmatize)):
        # first letter in low case for the lemmatize fonction parameter
        wntag = tag[0].lower()
    
        if tagFilter:
            wntag = wntag if wntag in tagFilter else None
        else:
            wntag = wntag if wntag in ['r', 'n', 'v'] else None
            
        if wntag:
            lemma = wnl.lemmatize(word.lower(), wntag) 
        else:
            if tagFilter:
                continue
            else:
                lemma = word.lower()
                
        sentenceToReturn.append(lemma)
    
    if toSentence:
        return ' '.join(word for word in sentenceToReturn)
    else:
        return sentenceToReturn

In [ ]:
def contains_only_integers(tup):
    '''
    This function returns <True> if the tupple 
    input communiqué contains only <int>.
    Returns <False> in all other cases.
    '''
    for item in tup:
        if not isinstance(item, int):
            return False
    return True

In [ ]:
def tuppleOfTwoInt(var=False, varName=''):
    '''
    Indicates whether the format of the argument <var> 
    is well communicated in the right format.
    The expected formats are: int or tupple of 2 int.
    If var is an int then the function returns 
    a tupple in the format (var,var) 
    If var is a tupple of 2 int, the function returns 
    the tuple <var> as is.
    Returns False without an error message if <var> is equal to False.
    Returns False with an error message indicating 
    that the format is not valid in all other cases.
    '''
    if varName:
        varName = str.strip(varName)+' '
    if isinstance(var, bool):
        if var:
            print(f'TuppleOfTwoInt : The format of the {varName}argument is incorrect')
        return False
    elif isinstance(var, int):
        return (var,var)    
    elif isinstance(var, tuple):
        if contains_only_integers(var):
            if len(var) == 2:
                return var    
    print(f'TuppleOfTwoInt : The format of the {varName}argument is incorrect')
    return False

In [ ]:
def preProcessingImage(img, resize=False, gray=False, gaus=False, gausKernel=5, equ=False):
    '''
    Returns an image <img>, previously processed.
    Processing may include:
     - Image resizing
     - Switching to grey level
     - Gaussian Blur
     - Equalizer
    The function accepts as input either 
    an image, or the path of an image.
    '''
    
    if isinstance(img, str):
        # We consider that it is not an image but the path of the image
        img = cv2.imread(img, cv2.IMREAD_COLOR)
    
    resize = tuppleOfTwoInt(var=resize, varName='resize')
    if resize:
        img = cv2.resize(img, resize)
    if gray:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if gaus and tuppleOfTwoInt(var=gausKernel, varName='gausKernel'):
        gausKernel = tuppleOfTwoInt(var=gausKernel, varName='gausKernel')
        img = cv2.GaussianBlur(img, gausKernel, cv2.BORDER_DEFAULT)
    if equ:
        img = cv2.equalizeHist(img)
    
    return img

In [ ]:
def descriptorExtraction(img):
    '''
    Returns the SIFT descriptors 
    of the image or of the image path <img> 
    communicated as an argument.
    '''
    
    if isinstance(img, str):
        # We consider that it is not an image but the path of the image
        img = cv2.imread(img, cv2.IMREAD_COLOR)
    
    img = preProcessingImage(img, gray=True, gaus=True, equ=True)
    sift = cv2.xfeatures2d.SIFT_create() # Création de l'objet SIFT
    _, des = sift.detectAndCompute(img,None)
    return des

In [ ]:
def groupAllDescriptors(dictData):
    '''
    Calculates a matrix containing all the descriptors
    calculated from a series of images.
    
    The images are read from the dictionary 
    communicated as argument with the key "path" that
    contains a list (or equivalent) of accesses to each image.
    
    The function creates the key "totalDes" containing 
    the descriptor matrix and then returns the dictionary.
    '''
    totalDes = []
    for img in dictData['path']:
        des = descriptorExtraction(img)
        totalDes.extend(des)
    dictData['totalDes'] = np.array(totalDes, dtype=np.int)
    return dictData

In [ ]:
def createKmeansOfBagOfVirtualWords(dictData):
    '''
    Returns a trained MiniBatchKMeans including 
    Labels represent virtual words 
    calculated from the matrix of descriptors 
    contained in the "totalDes" key of the dictionary 
    communicated as an argument.
    
    The MiniBatchKMeans is contained in 
    the "kmeans" key of the dictionary.
    
    The number of clusters is determined by 
    the square root of the number of descriptors.
    
    the batch size argument is equal to 3 times 
    the number of images in total.
    
    Returns the dictData dictionary.
    '''
    
    # definition of clustering parameters
    nClusters = int(round(np.sqrt(dictData['totalDes'].shape[0]),0))
    batchSize = dictData['path'].shape[0]*3

    print('nClusters =',nClusters)
    print('batchSize =',batchSize)

    dictData['kmeans'] = MiniBatchKMeans(n_clusters=nClusters,
                                         batch_size=batchSize).fit(dictData['totalDes'])
    return dictData

In [ ]:
def createMatrixOfImages(dictData, size=224):
    '''
    Creates a 4-dimensional matrix from 
    a list of images or path to the images.
    The 4 dimensions of the matrix are 
    distributed as below:
     - 1st dimension: Number of images
     - 2nd and 3rd dimension: Length and width of the image
     - 4th dimension: Depth of the image. 
        - Is equal to 3 if image is in color.
        - Is equal to 1 if image is grayscale.
    The function expects a dictionary as argument.
    The communicated images must be present 
    in the "path" key of the dictionary.
    This function is intended to return a matrix 
    that will be used as input for a VGG16 neural network.
    This neural network expects as input fixed 
    size images, by default square images of 224px side.
    This value can be modified with the argument <size>.
    The array is created and added to the 
    dictionary in the key "arrayImg".
    The function returns the dictionary.
    '''
    listImages = []
    
    for img in dictData['path']:
        img = preProcessingImage(img, resize=size)
        img = preprocess_input(img)
        listImages.append(img)
    print(f'Dimensions of the image matrix: {np.array(listImages).shape}')
    dictData['arrayImg'] = np.array(listImages)
    return dictData

In [ ]:
def createHistograms(dictData):
    '''
    This function calculates the normalized histograms 
    of the images (or path of the images) contained 
    in the "path" key of the dictionary communicated as argument.
    The images are listed in a List or equivalent.
    The normalization is performed from the 1st dimension 
    of the matrix containing the set of descriptors.
    This matrix is stored in the "totalDes" key of the dictionary.
    A numpy array containing the histograms is created 
    and added to the dictionary to the "histograms" key.
    The function returns the dictionary.
    '''
    list_histo = []
    for count, image in enumerate(dictData['path']):
        print(f'image processing number {count+1}/{dictData["path"].shape[0]}')
        # Vector of 0 equal in size to the number of unique labels in kmeans
        histo = np.zeros(len(set(dictData['kmeans'].labels_)))
        # Number of key points, for Histogram Normalisation
        nkp = dictData['totalDes'].shape[0] 
        
        for d in descriptorExtraction(image):
            idx = dictData['kmeans'].predict([d])
            histo[idx] += 1/nkp
        list_histo.append(histo)
    dictData['histograms'] = np.array(list_histo)
    print(f'Dimensions of the histogram matrix: {dictData["histograms"].shape}')
    return dictData

In [ ]:
def reductionTwoDimensionPCAandTSNE(dictData, col='histograms', pcaComponents=0.99):
    '''
    This function applies a PCA transformation 
    followed by a T-SNE transformation to a data set.
    The choice of PCA preprocessing is chosen to reduce 
    T-SNE computation time.
    
    The function expects a dictionary as argument.
    The data to be processed can be accessed with 
    the key <col> of the dictionary.
    The key <col> is given as argument.
    
    By default, PCA keeps 99% of the variance.
    It is possible to determine the parameter 
    <n_components> of PCA with the argument <pcaComponents>.
    
    T-SNE is set to return the data set from 
    the PCA transformation, with only 2 output dimensions.
    
    The result is added to the dictionary with the key "tsne".
    The function returns the dictionary.
    '''
    
    from scipy.sparse import issparse
    
    print(f'Treatment of the column {col}')
    print(f'Dimension of the original data set: {dictData[col].shape}')
    print('Start of PCA processing:')
    
    
    if pcaComponents <= 1:
        print(f'Variance preserved: {pcaComponents*100}%')
    else:
        print(f'Number of components conserved: {pcaComponents}')
        
    pca = PCA(n_components=pcaComponents)
    
    if issparse(dictData[col]):
        X_pca = pca.fit_transform(dictData[col].toarray())
    else:
        X_pca = pca.fit_transform(dictData[col])
    
    print(f'Data set size after PCA processing: {X_pca.shape}')
    print('End of PCA processing.')
    print(f'Start T-SNE processing:')
    
    tsne = TSNE(n_components=2)
    X_tsne = tsne.fit_transform(X_pca)
    
    print(f'Size of the dataset after T-SNE processing: {X_tsne.shape}')
    print(f'End of T-SNE processing.')
    
    dictData['tsne'] = X_tsne
    
    return dictData

In [ ]:
def displayTheFourImagesAndTheirHistograms(dictData, idx):
    '''
    Displays the same image 4 times in 
    succession with its associated histogram.
    The displayed image will be read from the index <idx>. 
    of the "path" key of the dictionary 
    communicated as an argument.
    Each image is displayed with a different 
    treatment in this order:
     1) No processing
     2) GrayScale
     3) GrayScale + Gaussian Blur
     4) GrayScale + Gaussian Blur + Equalization
     To display an image, the function calls 
     the <drawImageAndHistogram> function 
     with the appropriate parameters.
    '''
    drawImageAndHistogram(dictData,idx=idx,preprocessing=[])
    drawImageAndHistogram(dictData,idx=idx,preprocessing=['gray'])
    drawImageAndHistogram(dictData,idx=idx,preprocessing=['gray','gaus'])
    drawImageAndHistogram(dictData,idx=idx,preprocessing=['gray','gaus','equ'])

In [ ]:
def drawImageAndHistogram(dictData,idx=0,preprocessing=[]):
    '''
    Displays an image and its associated histogram.
    The displayed image will be read from the index <idx> 
    of the "path" key of the dictionary given as argument.
    Before displaying an image, a preprocessing list 
    can be applied to it.
    The list of preprocessing to be performed must 
    be given in the argument <preprocessing>.
    The list of possible arguments are:
     - "gray" --> "Gray Scale"
     - "gaus" --> "Gaussian Blur"
     - "equ" --> "Equalize Histogram"
    The title of the created image indicates 
    the list of preprocessing performed or 
    indicates "None" if no preprocessing has been chosen.
    '''
    gray=gaus=equ=False
    
    if 'gray' in preprocessing:
        gray=True
    if 'gaus'in preprocessing:
        gaus=True
    if 'equ' in preprocessing:
        equ=True

    # Matching the terms of the transformation to display the graph title
    dictTransformations={'gray':'Gray Scale',
                         'gaus':'Gaussian Blur',
                         'equ': 'Equalize Histogram'}
    # We add an 's' to 'transformation' in the title 
    # of the graph if we apply several transformations
    if len(preprocessing) > 1:
        s='s'
    else:
        s=''
    # Formatting of applied transformations
    if preprocessing == []:
        listTransformations = 'None'
    else:
        listTransformations = ' + '.join({ preprocessing: dictTransformations[preprocessing]\
                                          for preprocessing in preprocessing }.values())
        
    
    img = preProcessingImage(dictData['path'][idx],
                             gray=gray,
                             gaus=gaus,
                             gausKernel=5,
                             equ=equ)

    
    
    # Creation of the graph title
    graphTitle = ''.join([dictData['path'][idx].split('/')[-1],
                          '\nTransformation'+s+': ',listTransformations])
    
    fig = plt.figure(figsize=(20,10))
    fig.suptitle(graphTitle, fontsize=14)
    ax1 = fig.add_subplot(121)
    ax1.title.set_text('Analyzed image')
    ax1.imshow(img, cmap='gray')
    ax2 = fig.add_subplot(122)
    ax2.title.set_text('Histogram')
    ax2.hist(img.ravel(),256,[0,256])
    plt.show()

In [ ]:
def displayTSNESortedByColor(dictData,hue='category',figsize=(20,10)):
    '''
    Displays a scatterplot of a 2-dimensional matrix 
    contained in the key "tsne" of the dictionary 
    communicated as argument.
    The idea is to display the result of a T-SNE 
    dimension reduction in 2 dimensions.
    The color of the points is a function of the 
    argument <hue> communicated in argument and 
    refers to a key of the dictionary.
    '''
    plt.figure(figsize=figsize)
    plt.title('Product classified by colors according to their '+hue,
              fontsize=24)
    cmap = sns.color_palette("tab10",
                             n_colors=dictData[hue].nunique())
    sns.scatterplot(x=dictData['tsne'][:,0],
                    y=dictData['tsne'][:,1],
                    hue=dictData[hue],
                    palette=cmap)
    sns.color_palette("rocket_r",
                      as_cmap=True)
    plt.show()

In [ ]:
def createKMeansLabelsFromTSNE(dictData, n_clusters=7):
    '''
    This function calculates the Labels of a KMeans.
    The function expects a dictionary as argument.
    The calculated labels are added to the 
    dictionary with the key "labels".
    The number of clusters of the KMeans is 
    communicated as an entry in the argument "n_clusters".
    The data to be clustered are present 
    in the dictionary with the key "tsne".
    The idea of this function is to cluster 
    the data in 2 dimensions resulting from 
    a dimension reduction with T-SNE.
    Returns the dictionary.
    '''
    dictData['labels'] = pd.Series(KMeans(n_clusters=n_clusters)\
                                   .fit(dictData['tsne'])\
                                   .labels_,
                                   name='labels')
    return dictData

In [ ]:
def viewARIScore(dictData):
    '''
    Calculates and displays the ARI score of the data 
    coming from the "category" and "labels" keys of 
    the dictionary communicated as argument.
    '''
    print('ARI score between clustering according\nto categories and KMeans labels:',
          adjusted_rand_score(dictData['category'],
                              dictData['labels']))

In [ ]:
def showConfusionMatrix(dictData,
                        y_true='category',
                        y_pred='labels',
                        xlabel='Clusters',
                        ylabel='Product Categories'):
    from sklearn.metrics import confusion_matrix
    from sklearn.preprocessing import LabelEncoder
    
    # We guard against cases where y_true or y_pred are not numbers.
    le_true = LabelEncoder()
    le_pred = LabelEncoder()
    y_true= le_true.fit_transform(dictData[y_true])
    y_pred = le_pred.fit_transform(dictData[y_pred])
    
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8,8))
    sns.heatmap(cm,
                fmt="d",
                annot=True,
                linewidths=.5,
                xticklabels=le_pred.classes_,
                yticklabels=le_true.classes_)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.title('Confusion Matrix', fontsize=24)
    plt.show()

In [ ]:
def combineTwoMatricesInOne(dictData,
                            finalKey='vectors',
                            key1='tfidf',
                            key2='histograms'):
    '''
    This function joins 2 matrices into 1.
    The matrices are joined on their horizontal axis.
    Both matrices must have the same number of lines.
    The generated matrix has the same number of lines 
    in input as in output.
    The generated matrix has a number of columns equal 
    to the number of columns of the 1st matrix + the 
    number of columns of the 2nd matrix.
    The function expects a dictionary as argument.
    The matrices are present at the indexes of the 
    dictionary designated by the arguments <key1> and <key2>.
    The new generated matrix is stored in the 
    dictionary at the key <finalKey> given as argument.
    The function returns the dictionary.
    '''
    vectors = []
    
    print('Dimensions of two matrices:')
    print(f'   Matrix n°1 -- Name: {key1} -- Shape: {dictData[key1].shape}')
    print(f'   Matrix n°2 -- Name: {key2} -- Shape: {dictData[key2].shape}')
    
    for idx,vect in enumerate(dictData[key1]):
        vectors.append(np.concatenate((vect.toarray()[0],
                                       dictData[key2][idx])))
    dictData[finalKey] = np.array(vectors)
    print(f'Final matrix -- Name: {finalKey} -- Shape: {dictData[finalKey].shape}')
    return dictData

# 4. Import des données

In [ ]:
data = pd.read_csv('Flipkart/flipkart_com-ecommerce_sample_1050.csv')

In [ ]:
data.head(3).T

# 5. Analyse du DataSet

## 5.1 Types des données

In [ ]:
data.dtypes

## 5.2 Dimensions du DataFrame data

In [ ]:
data.shape

## 5.3 Valeurs manquantes

In [ ]:
data.isnull().sum()

## 5.4 Cardinalité de data

In [ ]:
data.nunique()

# 6. Classification des produits grâce <br />à leurs données textuelles descriptives

# 6.1 Préparation, Analyse des Features Textuelles

Dans cette partie nous allons **classer** <br />
et **regrouper** nos produits en autant de <br />
groupes qu'il existe de <u>catégories</u>. <br />
Nous réaliserons cela à travers <br />
notre **moteur de classification** à partir <br />
des <u>textes qui servent à les décrire</u>.

<u>L'objectif et d'obtenir du moteur <br />
de classification, le regroupement <br />
le plus proche par rapport au <br />
classement de référence initial</u>.

<u>Nous émettons l'hypothèse</u> qu'un **produit <br />
peut-être attribué à une catégorie à <br />
partir du vocubulaire utilisé dans sa description**.

<u>2 features permettent de décrire un produit</u> :
 - Le titre de l'annonce: "**product_name**"
 - La description de l'objet: "**description**"

Nous allons mettre en place le moteur de classification.<br />
Pour **optimiser ses performances**, nous allons devoir <br />
**pré-traiter les textes**.

L’objectif est, d’une part, de <u>débarrasser les textes</u> <br />
de tous les <u>éléments inutiles</u> qui n’apportent pas <br />
d’information permettant d’identifier le produit<br />
et d’autre part de *normaliser* le texte afin qu’un <br />
mot utilisé dans la description d’un produit ait <br />
exactement la même orthographe dans les autres <br />
descriptions des autres produits (gestion de la casse,<br /> 
de l’accord en genre et en nombre, verbes conjugués <br />
convertis dans leur forme à l’infinitif, etc.)

Alors seulement nous pourrons créer <br />
un **bag of words** constitué des seuls éléments<br /> 
de vocabulaire ayant une importance pour <br />
décrire nos produits.

*Un bag of words nous sert à décrire un document.<br />
Chaque mot représente une feature, une caractéristique.<br />
L'ensemble de plusieurs caractéristiques permet de<br />
décrire un produit et ainsi déterminer sa catégorie.*

Notre moteur de classification attribuera à <br />
chaque produit un **vecteur de caractéristiques**, <br />
qui sera fonction des **termes descriptifs** inclus <br />
dans sa description et **pondéré** avec l’ensemble <br />
des termes existants pour décrire <br />
l’ensemble des produits de notre jeu de données.

Pour **améliorer les performances** et également <br />
permettre de **visualiser la clusterisation des produits** <br />
de notre moteur de classification, nous procèderons <br />
à une <u>réduction en 2 dimensions</u> des vecteurs de caractéristiques.

Via une **clusterisation**, notre moteur **regroupera** ensemble <br />
les produits qu’il estimera **appartenir à la même catégorie**.

Enfin, nous pourrons **évaluer** notre moteur de <br />
classification en **comparant** le regroupement de <br />
l’ensemble des produits réalisé par notre moteur de <br />
classification avec la catégorie réelle de chaque produit.

### 6.1.1 Sélection des features à traiter

<u>Seules 3 features nous interresse dans cette partie</u> :
 - '**product_name**' correspond au titre du produit
 - '**description**' comme son nom l'indique, à sa description
 - '**product_category_tree**' renseigne sur la catégorie du produit.<br />
   Ce champ est précis et contient l'ensemble des catégories<br />
   et sous catégories associées au produit et <u>n'est pas exploitable en l'état</u>.

In [ ]:
df = data.copy()[['product_name','description','product_category_tree']]
df.head(2)

### 6.1.2 Fusion des colonnes 'product_name' et 'description'

Je fusionne le titre et la description du produit<br />
pour ne garder qu'un <u>unique champ</u> contenant<br />
l'<u>ensemble des termes</u> décrivant l'objet.

In [ ]:
df['title_desc'] = df['product_name']+' '+df['description']

### 6.1.3 Traitement de la main category

Chaque produit est caractérisé par la <br />
catégorie qui lui est associé.<br />
Il existe plusieurs niveaux de catégorie.<br />
Pour l'exercice nous n'allons <u>conserver <br />
que la **catégorie principale**</u> de chaque produit.

#### 6.1.3.1 Extraction de la main category

Création de la feature "category" qui contient <br />
le nom de la catégorie principale de chaque produit :

In [ ]:
df['category'] = df['product_category_tree']\
                   .apply(lambda row: first_product_category_tree(row))
df.drop('product_category_tree',
        axis=1,
        inplace=True)
df.head(2)

#### 6.1.3.2 Statistique sur la main category

##### 6.1.3.2.1 Nombre d'articles par catégorie

<u>Les produits sont parfaitements répartis entre les catégories</u> :

In [ ]:
df['category'].value_counts()

##### 6.1.3.2.2 Nombre de mots en moyenne par description  et par titre

<u>Je crée 3 colonnes contenant<br />
    le nombre de mots dans les features</u> :
 - "**description**"
 - "**product_name**"
 - "**title_desc**"

In [ ]:
# Assuming that a sentence with n words has n-1 spaces in it
df['count_description'] = df['description']\
                            .apply(lambda row: row.count(' ') + 1)
df['count_title'] = df['product_name']\
                            .apply(lambda row: row.count(' ') + 1)
df['count_title_description'] = \
        df['title_desc'].apply(lambda row: row.count(' ') + 1)

In [ ]:
cat = df.groupby('category').mean()[['count_description',
                                     'count_title',
                                     'count_title_description']]

In [ ]:
plt.figure(figsize=(20,10))

sns.lineplot(x=cat.index,
             y=cat['count_description'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='description')

sns.lineplot(x=cat.index,
             y=cat['count_title'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='title')

sns.lineplot(x=cat.index,
             y=cat['count_title_description'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='description + title')

plt.legend()
plt.xlabel('Category',fontsize=16)
plt.xticks(rotation=20,ha='right')
plt.ylabel('Number of words',fontsize=16)
plt.title('Number of words per category', fontsize=24)
plt.grid()
plt.show()

Le nombre de mots utilisés en moyenne<br />
pour décrire un produit est <u>inégalement <br />
réparti</u> entre les catégories, pouvant <br />
aller pratiquement du <u>simple au double</u>.

## 6.2 Traitement du texte

Cette étape est très importante pour l'**optimisation** <br />
de notre moteur de classification.

Nous allons **nettoyer** et **normaliser** nos descriptions <br />
en effectuant un ensemble d'opérations détaillées ci-dessous.

Pour cette étape, j'utilise la bilibliothèque **texthero**.

### 6.2.1 Nettoyage et Normalisation du texte

**Utilisation de text hero**:

<u>La méthode clean effectue le traitement suivant</u> :
1. texthero.preprocessing.**fillna()**
2. texthero.preprocessing.**lowercase()**<br />
&emsp;- *Passe la chaîne de caractère en minuscule*
3. texthero.preprocessing.**remove_digits()**<br />
&emsp;- *Remplace tous les digits par un seul espace*
4. texthero.preprocessing.**remove_punctuation()**<br />
&emsp;- *Remplace tous les signes de <br />
&emsp;&ensp;ponctuation par un seul espace*
5. texthero.preprocessing.**remove_diacritics()**<br />
&emsp;- *Supprime tous les diacritiques et les accents*
6. texthero.preprocessing.**remove_stopwords()**<br />
&emsp;- *Supprime les stopwords issues <br />
&emsp;&ensp;d'une liste de 179 stopwords de <br />
&emsp;&ensp;la bibliothèque NLTK.*
7. texthero.preprocessing.**remove_whitespace()**<br />
&emsp;- *Supprime les espaces inutiles en début et <br />
&emsp;&ensp;en fin de chaîne, les sauts de ligne, <br />
&emsp;&ensp;les tabulations et toute forme d'espace*

<u>J'affiche un exemple de description, <br />
**avant** et **après** traitement</u> :

<u>**Avant** Normalisation</u> :

In [ ]:
print(f'Number of words : {df.title_desc[0].count(" ") + 1}\n')
print(df.title_desc[0])

In [ ]:
df['title_desc_clean'] = df['title_desc'].pipe(hero.clean)

<u>**Après** Normalisation</u> :

In [ ]:
print(f'Number of words : {df["title_desc_clean"][0].count(" ") + 1}\n')
print(df['title_desc_clean'][0])

### 6.2.2 Lemmatisation du texte et<br />Filtrage des Noms + Verbes

Le processus de « **lemmatisation** » consiste<br />
à représenter les mots (ou « lemmes ») <br />
sous leur forme *canonique*.<br />
Par exemple pour un verbe, ce sera son infinitif.<br />
Pour un nom, son masculin singulier.<br />
L'idée étant encore une fois de ne **conserver <br />
que le sens des mots** utilisés dans le corpus.

De plus, dans notre objectif de ne <u>conserver que<br />
des features permettant de décrire un objet</u>,<br />
nous allons filtrer les résultats pour ne conserver <br />
que les **Noms** et **Verbes** des descriptions.

In [ ]:
df['title_desc_lemma'] = \
    df['title_desc_clean']\
        .apply(lambda row: \
               lemma_filter_english_words(row,
                                          tagFilter=['n',
                                                     'v']))

<u>**Avant** lemmatisation</u> :

In [ ]:
print(f'Number of words : {df.loc[0,"title_desc_clean"].count(" ") + 1}\n')
df.loc[0,'title_desc_clean']

<u>**Après** lemmatisation</u> :

In [ ]:
print(f'Number of words : {df.loc[0,"title_desc_lemma"].count(" ") + 1}\n')
df.loc[0,'title_desc_lemma']

### 6.2.3 Graphique Nombre de Mots par Catégorie par Traitement

In [ ]:
df.columns

In [ ]:
# Assuming that a sentence with n words has n-1 spaces in it
df['count_description'] = df['description']\
                            .apply(lambda row: row.count(' ') + 1)
df['count_title'] = df['product_name']\
                            .apply(lambda row: row.count(' ') + 1)
df['count_title_description'] = \
        df['title_desc'].apply(lambda row: row.count(' ') + 1)

In [ ]:
# Assuming that a sentence with n words has n-1 spaces in it
df['count_title_desc_clean'] = \
    df['title_desc_clean'].apply(lambda row: row.count(' ') + 1)
df['count_title_desc_lemma'] = \
    df['title_desc_lemma'].apply(lambda row: row.count(' ') + 1)

In [ ]:
cat = df.groupby('category').mean()[['count_title_description',
                                     'count_title_desc_clean',
                                     'count_title_desc_lemma']]

In [ ]:
plt.figure(figsize=(20,10))

sns.lineplot(x=cat.index,
             y=cat['count_title_description'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='without processing')

sns.lineplot(x=cat.index,
             y=cat['count_title_desc_clean'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='after cleaning ')

sns.lineplot(x=cat.index,
             y=cat['count_title_desc_lemma'],
             linewidth=3,
             marker='o',
             markersize=20,
             label='after cleaning and lemmatization')

plt.legend()
plt.xlabel('Category',fontsize=16)
plt.xticks(rotation=20,ha='right')
plt.ylabel('Number of words',fontsize=16)
plt.title('Number of words per category per preprocessing', fontsize=24)
plt.grid()
plt.show()

In [ ]:
100 - round(cat.count_title_desc_lemma.mean()/
      cat.count_title_description.mean()*100,2)

En moyenne, les étapes de **cleaning** et <br />
de **lematisation** suppriment **49.61%** <br />
de <u>mots inutiles à la description de nos produits</u>.

### 6.2.4 Création du dictionnaire dictTxt

A partir de maintenant nous allons <br />
manipuler différents objets.

Pour plus de simplicité, <br />
j'utiliserai un **dictionnaire** <br />
où <u>chaque clé fera référence <br />
à l'objet utilisé</u>.

J'utiliserai également un ensemble <br />
de **fonctions** que j'ai écrit et dont <br />
vous trouverez les **déclarations** en <br />
<u>première partie</u> de ce document.

Le plus souvent, <u>ces fonctions prennent <br />
en argument un dictionnaire</u> et nous <br />
retourne un dictionnaire enrichi <br />
d'une nouvelle clé contenant l'objet <br />
désiré (matrice tf-idf, t-sne, etc.).

J'ai essayé de choisir des noms <br />
de fonction explicites, n'hésitez pas<br />
à lire leur docstring si besoin.

In [ ]:
dictTxt = {}

J'ajoute au dictionnaire la key "**category**" <br />
contenant la feature "**category**" du DataFrame **df**.

In [ ]:
dictTxt['category'] = df.category

### 6.2.5 Création des vecteurs pondérés avec TF-IDF

Nous pouvons maintenant créer notre **bag of words**.<br />
Il s'agit du sac de mots, <u>sans ordre</u>, contenant <br />
<u>l'ensemble du vocabulaire</u> conservé et <br />
qui décrit au mieux nos produits.

Chaque produit de notre jeu de données est <br />
caractérisé par son <u>vocabulaire</u> et par le <br />
<u>nombre d'occurrences</u> de chacun de ses mots <br />
de vocabulaire.

Nous pourrions donc créer un **vecteur**, <br />
où chaque élément de ce vecteur correspondrait <br />
à un mot du **bac of words**, et où chaque valeur <br />
de chaque élément du vecteur correspondrait <br />
au nombre d'occurrences du mot dans <br />
la description du produit.

Cette approche est intéressante, on émet l'hypothèse <br />
que des catégories de produits rassemblent certains mots<br />
de vocabulaire spécifiques plutôt que d'autres.

Cependant, certains mots peuvent être fréquents <br />
et commun à plusieurs ou même à toutes les catégories.<br />
Nous allons donc *relativiser* l'occurrence d'un mot <br />
en fonction de sa présence dans l'ensemble <br />
des descriptions des produits.

Pour réaliser cela je vais utiliser **tf-idf** avec sklearn.

<u>Qu'est-ce que **TF-IDF**</u> :<br />
Term Frequency – Inverse Document Frequency <br />
est une mesure qui permet, à partir d’un <br />
ensemble de textes, de connaître l’importance <br />
relative de chaque mot.

Idéalement, un terme apparaît très fréquemment <br />
dans quelques textes seulement. <br />
Les mots qui apparaissent dans presque tous les <br />
documents ou très rarement n’ont que peu d’importance.

**TF-IDF** va nous permettre de créer une matrice de vecteurs,<br />
où chaque élement du vecteur représente un mot <br />
du **bag of word** et la valeur de chaque élément <br />
du vecteur représente l'importance d'un mot <br />
relativisé à l'ensemble des documents (ici, <br />
à l'ensemble des descriptions des produits <br />
du jeu de données.)

<u>Création de la matrice tf-idf</u> :

In [ ]:
dictTxt['tfidf'] = TfidfVectorizer()\
                     .fit_transform(df['title_desc_lemma'])

### 6.2.6 Réduction de dimension PCA / T-SNE

Afin nous permettre de <u>visualiser graphiquement</u> <br />
les **regroupements** de nos produits en fonction <br />
de leur description mais également d'**améliorer <br />
les performances** de notre moteur de classification, <br />
nous allons appliquer une **réduction de dimension** <br />
à notre matrice **tf-idf**.

L'objectif ici est de ramener nos vecteurs en **2 dimensions**.

Nous allons utiliser pour cela l'algorithme **T-SNE**.

Cependant le charge peut-être très <br />
lourde en termps de calcul.<br />
Pour éviter cette problématique,<br />
nous allons <u>appliquer une première <br />
réduction de dimension</u> en appliquant <br />
une <u>Analyse en Composante Principale</u>.<br />
Nous ferons le choix de conserver **99% <br />
de la variance** afin de ne pas sacrifier <br />
la qualité de nos données tout en <br />
réduisant drastiquement le temps necessaire <br />
à l'application de **T-SNE** pour la réduction <br />
en 2 dimensions de nos vecteurs TF-IDF.

In [ ]:
dictTxt = reductionTwoDimensionPCAandTSNE(dictTxt, col='tfidf')

<u>Nous pouvons afficher notre matrice TF-IDF<br />
maintenant réduite en 2 dimensions où <br />
chaque point représente un produit</u> :

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(dictTxt['tsne'][:,0], dictTxt['tsne'][:,1])
plt.title('TF-IDF matrix in 2 dimensions')
plt.show()

On peut d'ores et déjà remarquer <br />
des **regroupements** parmi les produits.

### 6.2.7 Affichage de la classification des descriptions selon leurs catégories

Ajoutons maintenant un peu de couleur.<br />
Chaque produit est maintenant représenté <br />
par une <u>couleur représentant sa catégorie</u><br />
parmi les 7 existantes.

In [ ]:
displayTSNESortedByColor(dictTxt,hue='category')

Certaines catégories semblent très bien regroupées <br />
comme "**Watches**" ou "**Beauty and Personal Care**".<br />
D'autres sont plus diffuses comme "**Home Furnising**".

On peut émettre l'hypothèse que <u>les groupes <br />
correctement taggés ont un vocubulaire très <br />
spécifique</u> alors que <u>les autres ont un vocubulaire <br />
plus général</u> susceptible d'être présent dans <br />
plusieurs catégories.

### 6.2.8 K-Means sur matrice T-SNE

L'idée à présent et de faire comme si <br />
nous ne connaissions pas la réelle <br />
catégorie des produits.

A partir de la matrice **T-SNE**, nous allons regrouper, <br />
via l'algorithme **K-Means** les points selon **7 clusters**.<br />
Chaque cluster représente l'une des 7 catégories.

In [ ]:
dictTxt = createKMeansLabelsFromTSNE(dictTxt)

### 6.2.9 Affichage de la classification des descriptions selon leurs clusters

<u>Affichons maintenant les produits selon leurs clusters K-Means</u> : 

In [ ]:
displayTSNESortedByColor(dictTxt,hue='labels')

Ici les clusters sont beaucoup plus <br />
nets et bien délimités que précédement.

On constate que certains clusters <br />
représentent assez fidèlement certaines <br />
vraies catégories, encore une fois <br />
comme "**Watches**" ou "**Beauty and Personal Care**".

Cependant les catégories les plus diffuses<br />
ont bien entendu des produits mal <br />
classés par **K-Means**.

### 6.2.10 Matrice de Confusion

Utilisons la **matrice de confusion** <br />
pour <u>visualiser les erreurs de classement</u>.

In [ ]:
showConfusionMatrix(dictTxt)

Les catégories ont la majorité de leurs produits<br />
qui sont correctements classés.<br />
Les clusters peuvent être attribués à leur catégorie <br />
respective en identifiant, pour chaque catégorie, <br />
le cluster qui la représente le plus.<br />
La catégorie "**Computers**" est la catégorie <br />
la plus difficile à classer. Presque la moitié des produits <br />
est attribuée à tort à la catégorie "**Beauty and Personal Care**."

### 6.2.11 Calcul du Score ARI

Pour mesurer l'erreur de notre moteur <br />
de classification, nous allons utiliser <br />
la métrique **ARI** via la fonction <br />
**adjusted_rand_score** de sklearn.

L'**adjusted_rand_score** calcule une mesure <br />
de similarité entre deux groupements <br />
en considérant toutes les paires d'échantillons <br />
et en comptant les paires qui sont assignées <br />
dans le même groupement ou dans des groupements <br />
différents dans les groupements prédits et réels.

*Le score ARI va de 0 pour un étiquetage aléatoire <br />
à 1 pour un étiquetage parfait*.

In [ ]:
viewARIScore(dictTxt)

Selon les essais nous obtenons <br />
un score variant de **0.44** à **0.56**.<br />
C'est un score très encourageant.

# 7. Classification des produits grâce à leurs images

Nous allons maintenant tenter de classer 
nos produits selon l'image associée à leur description.

Ici l'idée est similaire à l'analyse de texte :
 1. Obtenir des features des images,<br />
    à partir de leurs descripteurs
 2. Créer un **Bag of *Virtual* Words** <br />
    à partir de l'ensemble des features existantes
 3. Créer des <u>vecteurs de caractéristiques</u><br />
    à partir du **Bag of *Virtual* Words**
 4. Réaliser une <u>réduction de dimension</u> sur nos vecteurs
 5. <u>Clusteriser</u> notre matrice de vecteurs
 6. <u>Evaluer</u> le résultat de la clusterisation <br />
    de notre moteur de classification avec <br />
    la catégorie réelle associée à chaque image

<u>Je détaille ci-dessous 
deux approches</u> :
 - **SIFT** pour Scale-Invariant Feature Transform
 - **VGG16** qui est un modèle de réseau de neurones convolutifs

## 7.1 Méthode SIFT

La scale-invariant feature transform (**SIFT**), <br />
que l'on peut traduire par « transformation <br />
de caractéristiques visuelles invariante à l'échelle », <br />
est un <u>algorithme utilisé pour détecter et identifier <br />
les éléments similaires entre différentes images <br />
numériques</u> (éléments de paysages, objets, personnes, etc.). <br />
Il a été développé en 1999.

L'étape fondamentale de la méthode consiste <br />
à calculer ce que l'on appelle les « **descripteurs <br />
SIFT** » des images à étudier. <br />
Il s'agit d'informations numériques dérivées <br />
de l'analyse locale d'une image et qui caractérisent <br />
le contenu visuel de cette image de la façon <br />
la plus <u>indépendante possible de l'échelle</u> (« zoom » <br />
et résolution du capteur), <u>du cadrage, de l'angle <br />
d'observation et de l'exposition</u> (luminosité). 

Ainsi, **deux photographies d'un même objet** auront <br />
toutes les chances d'avoir des **descripteurs SIFT <br />
similaires**, et ceci d'autant plus si les instants <br />
de prise de vue et les angles de vue sont proches.

<u>Voici le protocole que l'on va mettre en place,<br />
pour créer notre moteur de classification des <br />
produits à partir des images avec **SIFT**</u> :

1. Pour chaque image du jeu de données
  - Lire l'image
  - Réaliser le **pre-processing** de l'image <br />
    *pour augmenter la capacité de SIFT à <br />
    détecter correctement les descripteurs*
   - Passer l'image en **niveau de gris** (par <br />
     souci de simplicité, les couleurs ne <br />
     changent pas fondamentalement le score final)
   - Application d'un **flou gaussien**
   - Egalisation de l'histogramme
   - **Extraction** des descripteurs
2. Création d'une matrice contenant <br />
   **l'ensemble des descripteurs**
3. **Clustérisation** pour création <br />
   des **Bag of *Virtual* Words**
4. Création des **Histogrammes**
5. Réduction de dimension **PCA** + **T-SNE**
6. Calcul du score **ARI**

### 7.1.1 Création du dictionnaire dictImages

De la même manière que la partie texte, <br />
j'utilise un **dictionnaire** pour stocker <br />
et manipuler nos données.

In [ ]:
dictImages = {}

### 7.1.2 Ajout du path des images...

J'importe le chemin complet (dossier + nom de fichier) <br />
de chaque image sous forme de **Series Pandas** pour <br />
qu'elles puissent êtres accessibles facilement.

In [ ]:
dictImages['path'] = data['image'].apply(lambda row: 'Flipkart/Images/'+row)

### 7.1.3 ...et de leurs catégories respectives

J'importe également sous forme de **Series Pandas** <br />
les catégories associées à chaque image.<br />
Comme les index sont les mêmes, l'ordre <br />
image/categorie est respecté.

In [ ]:
dictImages['category'] = df['category']

### 7.1.4 Calcul et regroupement de l'ensemble des descripteurs

On parcourt chacune des images referencées <br />
dans notre dictionnaire **dictImages** (chaque <br />
élement de la key "path") <br />

<u>Puis pour chaque image</u> :
 1. On charge l'image pour pouvoir travailler avec
 2. <u>On applique les pre-processing suivants</u> :
  1. Passage en niveau de gris
  2. Flou Gaussien
  3. Egalisation de l'Histogramme
 3. On applique SIFT à l'image pré-processée <br />
    pour extraire les descripteurs de l'image
 4. On ajoute les descripteurs à une matrice<br /> 
    destinée à contenir tous les descripteurs <br />
    de toutes les images
 5. On ajoute la matrice des descripteurs <br />
    à la key "**totalDes**" de notre dictionnaire.


In [ ]:
dictImages = groupAllDescriptors(dictImages)

In [ ]:
dictImages['totalDes'].shape

On obtient une matrice de **4 185 939 descripteurs**.<br />
Chaque descripteur est un vecteur de dimension (**1**,**128**).

### 7.1.5 A propos du pre-processing des images

J'affiche ci-dessous <u>1 image du jeu de données <br />
et son histogramme associé</u>, en appliquant <br />
successivement les opérations de pré-processing suivant:
 1. Aucun pré-processing
 2. Passage en niveau de gris
 3. Passage en niveau de gris + Flou Gaussien
  - Le Flou Gaussien nous sert à nettoyer l'image de son bruit.<br />
    Il évite à SIFT de détecter de mauvais descripteurs.<br />
 4. Passage en niveau de gris + Flou Gaussien + Egalisation de l'histogramme
  - L'égalisation de l'histogramme accentue <br />
    le contraste de l'image et améliore les performances <br />
    de SIFT dans la détection des descripteurs.

<u>Attention, dans cette partie j'évoque le terme <br />
***histogramme*** dans deux contextes différents</u> :

 - L'histogramme affiché ci-dessous correspond <br />
   à l'histogramme *classique* que l'on retrouve <br />
   par exemple dans les logiciels d'édition de photographie.
    - En photographie, l’histogramme nous permet de visualiser <br />
      comment se distribuent les tons clairs et foncés <br />
      dans notre image.<br />
      Autrement dit, il donne des informations sur <br />
      l’exposition de notre image.
      
      Plus simplement, à gauche de l’histogramme <br />
      sont représentés les pixels sombres, et à droite <br />
      les pixels clairs. <br />
      Plus il y a de pixels pour une tonalité <br />
      (très sombre, moyen, très clair, et tous les <br />
      intermédiaires), plus son “pic” sera élevé.
 - L'histogramme s'appuyant sur les **bac of *virtual* words** <br />
   (J'évoque ces histogrammes un peu plus bas dans cette partie.)<br />
   reprend le même principe que les histogrammes classiques.<br />
   Là où l'histogramme classique représente de gauche à droite <br />
   les tons clairs, moyens et foncés d'une image, <br />
   l'histogramme utilisé dans le contexte des **bag <br />
   of *virtual* words** représente quand à lui,<br />
   sur l'axe horizontal, l'ensemble des virtual words existants, <br />
   et sur l'axe vertical le nombre d'occurences des virtual words <br />
   au sein de l'image.<br />
   

In [ ]:
displayTheFourImagesAndTheirHistograms(dictImages,4)

### 7.1.6 Création du Bag of Virtual Words

Nous cherchons à trouver l'équivalent des mots,<br />
en tant que **features**, que nous avions pour <br />
l'analyse du texte.<br />
Pour retrouver l'équivalent des mots,<br /> 
nous allons créer des **mots virtuels**.

Grâce à l'algorithme **MiniBatchKMeans** <br />
(une variante de l'algorithme KMeans <br />
optimisé pour réduire le temps de calcul),<br />
nous allons clusteriser notre matrice <br />
de descripteurs calculée précedement.

Les descripteurs dont les caractéristiques <br />
seront proches seront naturellement regroupés <br />
dans des clusters identiques.

<u>Chaque centroïde sera alors considéré <br />
comme un mot virtuel</u>.

<u>La fonction **createKmeansOfBagOfVirtualWords** <br />
définie les **deux hyperparamètres** de <br />
**MiniBatchKMeans** de la manière suivante</u> : 

- <u>nClusters</u>: Le nombre de clusters
  - Est défini comme l'entier le plus proche <br />
    de la racine carrée du nombre total de descripteurs.
- <u>batchSize</u> : La taille des mini-lots<br />
  - Est défini comme 3 fois le nombre d'images

In [ ]:
dictImages = createKmeansOfBagOfVirtualWords(dictImages)

In [ ]:
dictImages['kmeans']

On obtient un **MiniBatchKMeans** entrainé avec **2046 clusters**.<br />
Chaque centroïde de ces clusters représente les *mots* <br />
de notre **Bag of Virtual Words**

### 7.1.7 Création des histogrammes

Nous allons maintenant créer les **histogrammes** <br />
de nos images.<br />
<u>Un histogramme est un vecteur de caractéristiques</u>.

Nous allons réaliser pour nos images le même genre <br />
d'opération qu'avec **TF-IDF** pour le texte de la <br />
partie précedente.

Un histogramme est un vecteur où chaque élement <br />
du vecteur correspond à un mot virtuel.<br />
La dimension de notre matrice d'histogrammes est <br />
donc égale à (Nombre d'images, Nombre de Clusters)<br />
soit ici (**1050**,**2046**).

<u>Pour renseigner chaque élement du vecteur d'une image <br />
nous réalisons les opérations suivantes</u> :

 - **Création d'un vecteur rempli de 0** <br />
   de dimension (1,Nombre_de_Clusters)
 - **Calcul des descripteurs des images** <br />
   selon le même procédé que vu précedement
 - Pour chaque descripteur:
  - Calcul d'un **predict** sur le **MiniBatrchKMeans** entrainé
  - Le label obtenu correspond au mot <br />
    *virtuel* associé à ce descripteur
   - Le Label correspond à l'indice <br />
     du vecteur histogramme
   - Par exemple si le label calculé pour <br />
     le 1er descripteur de la première image <br />
     est 3, alors nous renseignerons l'indice 3 <br />
     du vecteur de 0 crée précedement
  - On incrémente le vecteur de 0 associé <br />
    à l'image du descripteur calculé
   - Les histogrammes que nous créons sont **normalisés**.<br />
     Par conséquent, on incrémente l'histogramme <br />
     à l'indice correspondant de *1/Nombre_De_Descripteurs_Total*
  
Une fois l'opération réalisée pour <br />
tous les descripteurs d'une image, <br />
l'histogramme est finalisé.

Nous réalisons cette opération <br />
pour toutes les images.

Nous obtenons une matrice de <br />
dimension (**Nombre d'images**, **Nombre de mots virtuels** <br />
(c'est à dire de centroïdes dans le MiniBatchKMeans <br />
entrainé) ce qui correspond dans notre cas à (**1050**,**2046**)

In [ ]:
dictImages = createHistograms(dictImages)

### 7.1.8 Réduction de dimension PCA / T-SNE

Afin nous permettre de <u>visualiser graphiquement</u> <br />
les **regroupements** de nos produits en fonction <br />
de leur description mais également d'**améliorer <br />
les performances** de notre moteur de classification, <br />
nous allons appliquer une réduction de dimension <br />
à notre matrice de vecteurs.

L'objectif ici est de ramener nos vecteurs en **2 dimensions**.

Nous allons utiliser pour cela l'algorithme **T-SNE**.

Cependant la charge peut-être très <br />
lourde en temps de calcul.<br />
Pour éviter cette problématique,<br />
nous allons <u>appliquer une première <br />
réduction de dimension</u> en appliquant <br />
une <u>Analyse en Composante Principale</u>.<br />
Nous ferons le choix de conserver **99% <br />
de la variance** afin de ne pas sacrifier <br />
la qualité de nos données tout en <br />
réduisant drastiquement le temps nécessaire <br />
à l'application de **T-SNE** pour la réduction <br />
en 2 dimensions de nos histogrammes.

In [ ]:
dictImages = reductionTwoDimensionPCAandTSNE(dictImages)

<u>Nous pouvons afficher notre matrice d'histogrammes<br />
maintenant réduite en 2 dimensions où <br />
chaque point représente un produit</u> :

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(dictImages['tsne'][:,0], dictImages['tsne'][:,1])

A ce stade, aucun regroupement de points ne semble se démarquer.

### 7.1.9 Affichage de la classification des images selon leurs catégories

Ajoutons maintenant un peu de couleur.<br />
Chaque produit est maintenant représenté <br />
par une couleur représentant sa catégorie<br />
parmi les 7 existantes.

In [ ]:
displayTSNESortedByColor(dictImages,hue='category')

Il est très difficile de distinguer <br />
des regroupements selon les catégories.

### 7.1.10 K-Means sur matrice T-SNE

L'idée à présent et de faire comme si <br />
nous ne connaissions pas la réelle <br />
catégorie des produits.

A partir de la matrice **T-SNE**, nous allons regrouper, <br />
via l'algorithme **K-Means** les points selon **7 clusters**.<br />
Chaque cluster représente l'une des 7 catégories.

In [1]:
dictImages = createKMeansLabelsFromTSNE(dictImages)

NameError: name 'createKMeansLabelsFromTSNE' is not defined

### 7.1.11 Affichage de la classification des images selon leurs clusters

<u>Affichons maintenant les produits selon leurs clusters K-Means</u> : 

In [ ]:
displayTSNESortedByColor(dictImages,hue='labels')

Les points sont ici regroupées d'une manière beaucoup plus homogène.

### 7.1.13 Matrice de Confusion

Utilisons la **matrice de confusion** <br />
pour <u>visualiser les erreurs de classement</u>.

In [ ]:
showConfusionMatrix(dictImages)

Les catégories sont mal classées et il est impossible <br />
d'associer avec certitude une catégorie à un numéro de cluster.

### 7.1.12 Calcul du Score ARI

Pour mesurer l'erreur de notre moteur <br />
de classification, nous allons utiliser <br />
la métrique **ARI** via la fonction <br />
**adjusted_rand_score** de sklearn.

L'**adjusted_rand_score** calcule une mesure <br />
de similarité entre deux groupements <br />
en considérant toutes les paires d'échantillons <br />
et en comptant les paires qui sont assignées <br />
dans le même groupement ou dans des groupements <br />
différents dans les groupements prédits et réels.

*Le score ARI va de 0 pour un etiquetage aléatoire <br />
à 1 pour un étiquetage parfait*.

In [ ]:
viewARIScore(dictImages)

Nous obtenons un score d'environ **7.5%**<br />
Celà peut parraitre peu mais les images <br />
n'étaient vraiment pas optimisées ni en nombre suffisant.<br />
<u>Ce score peut-être amélioré</u>.

## 7.2 Méthode Transfert Learning avec VGG16

Nous allons dans cette partie utiliser **Keras**,<br />
une bibliothèque de **Deep Learning**.

Nous utiliserons une version du <u>réseau de neurones <br />
convolutif VGG-Net nommé **VGG16**</u>.

Le modèle **VGG16** atteint une précision de **92,7%** <br />
dans le top 5 des tests d'**ImageNet**.<br />
**ImageNet** est un ensemble de données de plus <br />
de <u>15 millions d'images haute résolution <br />
étiquetées</u> appartenant à environ **22 000 catégories**. 

**Keras** nous fourni une version pré-entraînée de **VGG16**.

Pour mettre en place notre moteur de classification <br />
d'images, nous appliquerons une technique <br />
nommée *Transfert Learning*.

Simplement, le *Transfert Learning* consiste <br />
à utiliser la connaissance déjà acquise <br />
par un modèle entraîné (ici **VGG16**) pour l'adapter <br />
à notre problématique.

Actuellement **VGG16** permet de classer une image <br />
parmi **1000 catégories** différentes.

Nous souhaitons donc pouvoir <u>utiliser <br />
la puissance de **VGG16**</u> mais <u>adaptée à <br />
notre problématique</u>. C'est à dire pouvoir <br />
<u>classer une image</u> non pas parmi 1000 catégories <br />
mais <u>parmi nos 7 catégories</u>.

### 7.2.1 Installer Keras

Avant d'aller plus loin,<br />
vous devrez installer Keras.

<u>Pour installer Keras</u> : 
 - Dans Anaconda Prompt
  - "conda install -c anaconda keras-gpu"

### 7.2.2 Explication rapide de VGG16

<u>Voici une vue des différentes couches de VGG16</u> :

![Représentation VGG16](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)

**VGG-16** est constitué de plusieurs couches, <br />
dont **13** couches de convolution et **3** fully-connected. <br />

<u>Il prend en entrée une image en couleurs de taille 224  × 224 px</u> <br />
et la classifie dans une des 1000 classes. <br />
<u>Il renvoie donc un vecteur de taille 1000</u>, <br />
qui contient les probabilités d'appartenance <br />
à chacune des classes. 

Dans notre approche *Transfert Learning*, nous n'allons <br />
pas utiliser la dernière couche **softmax** chargée de <br />
classer l'image parmi l'une des 1000 catégories.

Nous récupérerons, pour chaque image, le **vecteur** <br />
de dimension (**1**,**1**,**4096**), puis, comme pour la méthode SIFT,<br /> 
ou pour le traitement du texte que nous avons <br />
réalisé précedement dans ce projet, nous répéterons <br />
les étapes de :
 - Réduction de dimension 
 - Clustering 
 - Et enfin nous calculerons le score ARI <br />
   entre le groupement des produits selon <br />
   les catégories et les labels d'un <br />
   clustering KMeans réalisé sur la <br />
   projection en 2 dimensions de nos vecteurs.

### 7.2.3 Création du modèle pré-entrainé par Keras

Nous allons dans un premier temps <u>importer le modèle VGG16</u>.<br />
Puis nous créerons ensuite <u>notre propre modèle</u>, <br />
qui sera <u>une copie de VGG16 à l'exception de <br />
la dernière couche **softmax**</u>.

<u>Création du modèle VGG-16 implémenté par Keras</u> :

In [ ]:
model = VGG16(weights="imagenet",
              include_top=True, # test avec True
              input_shape=(224, 224, 3))

<u>On indique qu'on ne souhaite ré-entrainer <br />
aucune des couches du modèle</u> :

In [ ]:
for layer in model.layers:
   layer.trainable = False

<u>On définit le nouveau modèle en <br />
définissant son entrée et sa sortie</u> :

<u>En entrée</u>, nous voulons utiliser l'entrée du modèle VGG16<br />
<u>En sortie</u>, nous souhaitons obtenir le vecteur généré <br />
par VGG16 juste avant la dernière couche softmax, <br />
soit son **avant-dernière couche**.

In [ ]:
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

<u>Pour s'assurer que le nouveau modèle est <br />
bien configuré, j'affiche son résumé que <br />
je compare avec le modèle VGG16 initial</u> : 

<u>Résumé du modèle VGG16</u> :

In [ ]:
model.summary()

<u>Résumé de notre nouveau modèle </u>:<br />
On remarque que la dernière couche <br />
n'a pas été importée et que le modèle <br />
ne va pas être ré-entrainé, même partiellement.

In [ ]:
new_model.summary()

### 7.2.4 Création du dictionnaire dictModel

Comme précedement, j'utilise un dictionnaire <br />
pour stocker et manipuler nos données.

In [ ]:
dictModel={}

### 7.2.5 Ajout du path des images...

J'importe le chemin complet (dossier + nom de fichier) <br />
de chaque image sous forme de **Series Pandas** pour <br />
qu'elles puissent être accessibles facilement.

In [ ]:
dictModel['path'] = data.copy()['image'].apply(lambda row: 'Flipkart/Images/'+row)

### 7.2.6 ...et de leurs catégories respectives

J'importe également sous forme de **Series Pandas** <br />
les catégories associées à chaque image.<br />
Comme les index sont les mêmes, l'ordre <br />
image/catégorie est respecté.

In [ ]:
dictModel['category'] = df.copy()['category']

### 7.2.7 Préparation des images

<u>VGG16 attend en entrée une matrice d'images<br /> 
en 4 dimensions composée de la façon suivante</u> :
 - <u>1ère dimension</u> : Correspond au nombre d'images importées
  - 1050 dans notre cas
 - <u>2ème et 3ème dimension</u> : correspond à la largeur et hauteur de l'image
  - VGG16 attend en entrée des images de 224 x 224 px.
 - <u>4ème dimension</u> : Correspond à la profondeur de l'image
  - Vaut 3 si les photos sont en couleurs (notre cas)
  - Vaut 1 si les photos sont en noir et blanc

<u>Préparons les images au bon format <br />
pour notre Model basé sur VGG16</u> :
 1. Je crée un matrice d'images en 4 dimensions<br />
 2. Chaque image est préalablement redimensionnée <br />
    avec les dimensions 224 x  224 px.
 3. La matrice est stockée à l'intérieur du <br />
    dictionnaire sous la clé "**arrayImg**".

In [ ]:
dictModel = createMatrixOfImages(dictModel)

### 7.2.8 Extraction des histogrammes

Pour obtenir nos histogrammes avec notre <br />
nouveau modèle, il suffit d'utiliser la méthode<br />
**predict** en passant en argument la matrice <br />
d'images en 4 dimensions.

Le modèle nous retourne une matrice <br />
de (ici 1050) vecteurs, où chaque vecteur <br />
est de dimension (1, 4096).

In [ ]:
dictModel['histograms'] = new_model.predict(dictModel['arrayImg'])

In [ ]:
dictModel['histograms'].shape

### 7.2.9 Clustering PCA / T-SNE

Afin nous permettre de <u>visualiser graphiquement</u> <br />
les **regroupements** de nos produits en fonction <br />
de leur description mais également d'**améliorer <br />
les performances** de notre moteur de classification, <br />
nous allons appliquer une réduction de dimension <br />
à notre matrice de vecteurs.

L'objectif ici est de ramener nos vecteurs en **2 dimensions**.

Nous allons utiliser pour cela l'algorithme **T-SNE**.

Cependant la charge peut-être très <br />
lourde en temps de calcul.<br />
Pour éviter cette problématique,<br />
nous allons <u>appliquer une première <br />
réduction de dimension</u> en appliquant <br />
une <u>Analyse en Composante Principale</u>.<br />
Nous ferons le choix de conserver **99% <br />
de la variance** afin de ne pas sacrifier <br />
la qualité de nos données tout en <br />
réduisant drastiquement le temps nécessaire <br />
à l'application de **T-SNE** pour la réduction <br />
en 2 dimensions de nos histogrammes.

In [ ]:
dictModel = reductionTwoDimensionPCAandTSNE(dictModel)

<u>Nous pouvons afficher notre matrice d'histogrammes<br />
maintenant réduite en 2 dimensions où <br />
chaque point représente un produit</u> :

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(dictModel['tsne'][:,0], dictModel['tsne'][:,1])

Les produits semblent nettement plus répartis <br />
par groupe que lors de l'utilisation de SIFT.<br />
Même si à ce stade il n'est pas possible de <br />
savoir à quel point le moteur de classification <br />
sera efficace, cette première représentation <br />
avec notre model basé sur VGG16 semble plus <br />
efficace que la méthode SIFT.

### 7.2.10 Affichage de la classification des images selon leurs catégories

Ajoutons maintenant un peu de couleur.<br />
Chaque produit est maintenant représenté <br />
par une couleur représentant sa catégorie<br />
parmi les 7 existantes.

In [ ]:
displayTSNESortedByColor(dictModel,hue='category')

Comme pour le classement à partir de la description <br />
texte des images, certaines catégories semblent <br />
très bien regroupées comme "**Watches**" ou "**Beauty and Personal Care**".<br />
D'autres sont plus diffuses comme "**Home Furnising**".

On peut émettre l'hypothèse que les groupes <br />
correctement taggués ont des caractéristiques visuelles très <br />
spécifiques alors que les autres ont des caractéristiques <br />
plus générales et susceptibles d'être présentes dans <br />
plusieurs catégories.

### 7.2.11 K-Means sur matrice T-SNE

L'idée à présent et de faire comme si <br />
nous ne connaissions pas la réelle <br />
catégorie des produits.

A partir de la matrice **T-SNE**, nous allons regrouper, <br />
via l'algorithme **K-Means** les points selon **7 clusters**.<br />
Chaque cluster représente l'une des 7 catégories.

In [ ]:
dictModel = createKMeansLabelsFromTSNE(dictModel)

### 7.2.12 Affichage de la classification des images selon leurs clusters

<u>Affichons maintenant les produits selon leurs clusters K-Means</u> : 

In [ ]:
displayTSNESortedByColor(dictModel,hue='labels')

Ici les clusters sont beaucoup plus <br />
nets et bien délimités que précedement.

On constate que certains clusters <br />
représentent assez fidèlement certaines <br />
vraies catégories, encore une fois <br />
comme "**Watches**" ou "**Beauty and Personal Care**".

Cependant les catégories les plus diffuses<br />
ont bien entendu des produits mal <br />
classés par **K-Means**.

### 7.2.13 Matrice de Confusion

Utilisons la **matrice de confusion** <br />
pour <u>visualiser les erreurs de classement</u>.

In [ ]:
showConfusionMatrix(dictImages)

Certaines catégories ont leurs produits<br />
qui sont correctements classés.<br />
D'autres catégories n'ont pas pu être associées <br />
clairement à un cluster comme la catégorie "**Baby Care**".

Il est impossible d'associer avec certitude <br />
une catégorie à un numéro de cluster.

### 7.2.14 Calcul du Score ARI

Pour mesurer l'erreur de notre moteur <br />
de classification, nous allons utiliser <br />
la métrique **ARI** via la fonction <br />
**adjusted_rand_score** de sklearn.

L'**adjusted_rand_score** calcule une mesure <br />
de similarité entre deux groupements <br />
en considérant toutes les paires d'échantillons <br />
et en comptant les paires qui sont assignées <br />
dans le même groupement ou dans des groupements <br />
différents dans les groupements prédits et réels.

*Le score ARI va de 0 pour un étiquetage aléatoire <br />
à 1 pour un étiquetage parfait*.

In [ ]:
viewARIScore(dictModel)

Selon les essais nous obtenons <br />
un score variant de **0.45** à **0.50**.<br />
C'est un score très encourageant.

# 8. Fusion des features Textes et Images

Nous avons testé notre moteur de classification à partir <br />
des **textes de description** des produits puis de leurs **images**, <br />
nous allons tenter d'**améliorer l'efficacité de notre<br />
moteur de classification** en <u>fusionnant les vecteurs <br />
de caractéristiques</u> obtenus avec **TF-IDF** pour l'analyse textuelle,<br />
puis <u>les vecteurs issus de notre modèle basé sur **VGG16**</u><br />
pour l'analyse des images.

Nous obtiendrons ainsi, pour chaque produit, <u>un unique vecteur <br />
de caractéristiques</u> provenant à la fois des **features <br />
textes et images**.

Les étapes de réalisation seront exactement <br />
les mêmes que vu précedement.<br />
Je ne rentrerai donc pas dans les détails dans cette partie.

## 8.1 Création et initialisation du dictionnaire dictTxt

<u>Je crée un nouveau dictionnaire dans lequel j'importe</u> :
 - Les vecteurs de caractéristiques textuelles
 - Les vecteurs de caractéristiques des images <br />
   obtenus à partir du modèle basé sur VGG16
 - La liste des catégories associées à chaque produit.

In [ ]:
dictTxtImages = {}

In [ ]:
dictTxtImages['category'] = dictTxt['category']

In [ ]:
dictTxtImages['tfidf'] = dictTxt['tfidf']

In [ ]:
dictTxtImages['histograms'] = dictModel['histograms']

## 8.2 Fusion des vecteurs de caractéristiques de textes et d'images

Je combine les deux matrices de vecteurs en une seule <br />
et je stocke cette nouvelle matrice dans <br />
la clé "**vectors**" du dictionnaire **dictTxtImages**.

In [ ]:
dictTxtImages = combineTwoMatricesInOne(dictTxtImages)

## 8.3 Clustering PCA / T-SNE

<u>On effectue la réduction des vecteurs en 2 dimensions</u> :

In [ ]:
dictTxtImages = reductionTwoDimensionPCAandTSNE(dictTxtImages)

<u>Nous pouvons afficher notre matrice d'histogrammes<br />
maintenant réduite en 2 dimensions où <br />
chaque point représente un produit</u> :

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(dictTxtImages['tsne'][:,0], dictTxtImages['tsne'][:,1])

## 8.4 Affichage de la classification des produits selon leurs catégories

Ajoutons maintenant un peu de couleur.<br />
Chaque produit est maintenant représenté <br />
par une couleur représentant sa catégorie<br />
parmi les 7 existantes.

In [ ]:
displayTSNESortedByColor(dictTxtImages,hue='category')

Comme pour le classement à partir de la description <br />
texte des images, certaines catégories semblent <br />
très bien regroupées comme "**Watches**" ou "**Beauty and Personal Care**".<br />
D'autres sont plus diffuses comme "**Home Furnising**".

## 8.5 K-Means sur matrice T-SNE

L'idée est toujours de faire comme si <br />
nous ne connaissions pas la réelle <br />
catégorie des produits.

A partir de la matrice **T-SNE**, nous allons regrouper, <br />
via l'algorithme **K-Means** les points selon **7 clusters**.<br />
Chaque cluster représente l'une des 7 catégories.

In [ ]:
dictTxtImages = createKMeansLabelsFromTSNE(dictTxtImages)

## 8.6 Graphique Description/Cluster

<u>Affichons maintenant les produits selon leurs clusters K-Means</u> : 

In [ ]:
displayTSNESortedByColor(dictTxtImages,hue='labels')

## 8.7 Matrice de Confusion

Utilisons la **matrice de confusion** <br />
pour <u>visualiser les erreurs de classement</u>.

In [ ]:
showConfusionMatrix(dictImages)

Certaines catégories ont leurs produits<br />
qui sont correctements classés.<br />
D'autres catégories n'ont pas pu être associées<br />
clairement à un cluster comme la catégorie "**Baby Care**<br />
ou la catégorie "**Kitchen & Dining**".

Il est impossible d'associer avec certitude<br />
une catégorie à un numéro de cluster.

## 8.8 Calcul du Score ARI

Pour mesurer l'erreur de notre moteur <br />
de classification, nous allons utiliser <br />
la métrique **ARI** via la fonction <br />
**adjusted_rand_score** de sklearn.

L'**adjusted_rand_score** calcule une mesure <br />
de similarité entre deux groupements <br />
en considérant toutes les paires d'échantillons <br />
et en comptant les paires qui sont assignées <br />
dans le même groupement ou dans des groupements <br />
différents dans les groupements prédits et réels.

*Le score ARI va de 0 pour un étiquetage aléatoire <br />
à 1 pour un étiquetage parfait*.

In [ ]:
viewARIScore(dictTxtImages)

Nous obtenons un score de même ordre de grandeur <br />
qu'avec utilisation seul du moteur de classification <br />
textuel ou par image avec le model basé sur VGG16.

# 9. Conclusion

Nous avons successivement tenté de classer<br />
nos produits à partir de leurs données<br />
descriptives **textuelles** ainsi qu'à partir<br />
de leurs **images**.<br />
Nous avons ensuite tenté de classer les <br />
produits en <u>**fusionnant** les données textes <br />
et images</u>.

Les tests ont été réalisés à partir d'un <br />
petit échantillon de **1050** produits.

A ce stade la classification à partir <br />
des données textes (titre + description) <br />
donne les résultats les plus satisfaisants.<br />
L'analyse d'images n'apporte pas de précision <br />
supplémentaire.

Cependant, <u>la précision du moteur de classification<br /> 
basée sur l'analyse des images peut être grandement <br />
améliorée</u> si nous décidons, par exemple, de ré-entrainer <br />
un modèle basé sur **VGG16** à partir d'une base <br /> 
d'images correspondant à nos catégories de produits.

Globalement, les résultats sont très **encourageants** <br />
et nous permettent de donner <u>un avis **positif** <br />
sur la faisabilité du moteur de classification</u>.